# Step 11: Multi-Agent Workflow - NASA Slideshow Builder

This notebook demonstrates how to build a **multi-agent workflow** using the Microsoft Agent Framework.

## What We're Building

A collaborative AI system that creates NASA-themed presentation slideshows on any topic by:

1. **Planner Agent** - Creates a structured presentation outline
2. **Researcher Agent** - Finds suitable NASA images via API
3. **Reviewer Agent** - Validates that images match the outline
4. **Judge Agent** - Final arbiter when max retries reached

## Architecture

```
User Request
     │
     ▼
┌──────────┐    Outline    ┌────────────────────────────────────────┐
│ Planner  │──────────────▶│   Per-Slide Workflow (×N slides)       │
│  Agent   │               │                                        │
└──────────┘               │  NASA API ──▶ Researcher ──▶ Reviewer  │
                           │       ▲                       │        │
                           │       └───── retry loop ◀─────┘        │
                           │                                        │
                           │  Judge (if max attempts reached)       │
                           └────────────────────────────────────────┘
                                           │
                                           ▼
                                  Final Slideshow
```

## Use Cases

- **Planetary Tour**: Explore the solar system's planets
- **Apollo 11 Mission**: Historic Moon landing moments  
- **Space Shuttle Program**: 30 years of shuttle missions
- **Hubble Discoveries**: Amazing images from space telescopes

## Setup & Configuration

In [ ]:
%pip install agent-framework --pre

In [ ]:
%pip install -q --pre azure-identity httpx pydantic

In [ ]:
import os
import sys
from pathlib import Path
from IPython.display import display, Markdown

# Add parent directory to path so we can import 11-agent-workflow as a package
notebook_dir = Path.cwd()
parent_dir = notebook_dir.parent
if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

# Load .env from parent directory (same approach as lab-7c)
env_path = Path("../.env")
if env_path.exists():
    for line in env_path.read_text().splitlines():
        if '=' in line and not line.startswith('#'):
            key, value = line.split('=', 1)
            os.environ[key.strip()] = value.strip()

# Landing Zone settings (from Lab 1a)
APIM_URL = os.environ.get("APIM_URL", "")
APIM_KEY = os.environ.get("APIM_KEY", "")
MODEL_NAME = os.environ.get("MODEL_NAME", "gpt-4.1-mini")

if not APIM_URL or not APIM_KEY:
    print("❌ Missing APIM_URL or APIM_KEY in .env file!")
    print("   Please complete Lab 1a first")
else:
    display(Markdown(f'''
### ✅ Configuration Loaded from `.env`

| Setting | Value |
|---------|-------|
| APIM Gateway | `{APIM_URL[:50]}...` |
| Model | `{MODEL_NAME}` |
| API Key | `***{APIM_KEY[-4:]}` |
'''))

## Test NASA Images API

First, let's verify the NASA Images API is working. This is a free, public API with no authentication required.

In [25]:
from nasa_api import search_nasa_images, format_images_summary

# Test search for Mars images
mars_images = await search_nasa_images("Mars Curiosity rover", page_size=5)

print(f"🔍 Found {len(mars_images)} Mars images\n")
print(format_images_summary(mars_images))

🔍 Found 5 Mars images

1. [PIA15106] Head of Mast on Mars Rover Curiosity
   Keywords: Mars Science Laboratory MSL
   Description: This view of the head of the remote sensing mast on the Mars Science Laboratory mission rover, Curiosity, shows seven of the 17 cameras on the rover....

2. [PIA17068] Curiosity Mars Rover Drilling Into Its Second Rock
   Keywords: Mars, Mars Science Laboratory MSL
   Description: This frame from an animation from NASA Mars rover Curiosity shows the rover drilling into rock target Cumberland. The drilling was performed during th...

3. [PIA14253] Mars Rover Curiosity Raising Turret
   Keywords: Mars Science Laboratory MSL
   Description: This photograph of the NASA Mars Science Laboratory rover, Curiosity, was taken during testing on June 3, 2011. The turret at the end of Curiosity rob...

4. [PIA13809] Top of Mars Rover Curiosity Remote Sensing Mast
   Keywords: Mars Science Laboratory MSL
   Description: The remote sensing mast on NASA Mars rover Curiosit

In [26]:
# Display a sample image thumbnail
from IPython.display import display, HTML

if mars_images and mars_images[0].thumbnail_url:
    img = mars_images[0]
    display(HTML(f"""
    <div style="text-align: center; padding: 20px;">
        <img src="{img.thumbnail_url}" style="max-width: 400px; border-radius: 8px; box-shadow: 0 4px 12px rgba(0,0,0,0.3);">
        <p style="color: #666; margin-top: 10px;"><strong>{img.title}</strong></p>
        <p style="color: #888; font-size: 0.9em;">NASA ID: {img.nasa_id}</p>
    </div>
    """))

## Understanding the Workflow

### The Workflow Graph

Our workflow follows a **search → select → review** pattern with retry loops:

```python
# Simplified workflow structure
search → select → review → [approved? → DONE : retry → search]
                         → [max_attempts? → judge → DONE]
```

### Key Components

| Component | File | Purpose |
|-----------|------|--------|
| `SlideWorkflowState` | `state.py` | Shared state between all executors |
| `SearchExecutor` | `executors.py` | Queries NASA API |
| `SelectExecutor` | `executors.py` | LLM picks best image |
| `ReviewExecutor` | `executors.py` | LLM validates selection |
| `JudgeExecutor` | `executors.py` | Final arbiter |

### Agent Framework Concepts

- **Executor**: A node in the workflow graph that processes state
- **WorkflowBuilder**: Connects executors with conditional edges
- **WorkflowContext**: Provides `send_message()` and `yield_output()` for state flow

In [ ]:
# Let's look at how the workflow is built
from workflow import create_slideshow_workflow

# View the workflow definition
import inspect
print(inspect.getsource(create_slideshow_workflow))

## Run the Multi-Agent Workflow

Now let's run the complete workflow to build a NASA slideshow!

In [28]:
from agents import SlideshowOrchestrator
from display_helpers import (
    display_outline, 
    display_slide_progress, 
    display_selection,
    display_review,
    display_final_slideshow,
    display_event
)

# Initialize the orchestrator with APIM configuration from .env
orchestrator = SlideshowOrchestrator(
    apim_endpoint=APIM_URL,
    apim_key=APIM_KEY,
    deployment_name=MODEL_NAME,
)

print("✅ Orchestrator initialized with 4 agents:")
print("   • PlannerAgent - Creates presentation outline")
print("   • ResearcherAgent - Selects NASA images")
print("   • ReviewerAgent - Validates image relevance")
print("   • JudgeAgent - Final arbiter when needed")
print(f"\n🔗 Using APIM Gateway with model: {MODEL_NAME}")

✅ Orchestrator initialized with 4 agents:
   • PlannerAgent - Creates presentation outline
   • ResearcherAgent - Selects NASA images
   • ReviewerAgent - Validates image relevance
   • JudgeAgent - Final arbiter when needed

🔗 Using APIM Gateway with model: gpt-4.1-mini


### Phase 1: Generate Presentation Outline

The Planner Agent creates a structured outline based on your request.

In [29]:
# Define what kind of presentation we want
# You can modify this to any NASA-related topic!
USER_REQUEST = """
Create a visual tour of our solar system's planets and the sun.
Show each planet from far as a circle. 
Start with the sun.
In each title put a the text "FULL PLANET VISIBLE"
"""

print("📝 Generating presentation outline...")
outline = await orchestrator.generate_outline(USER_REQUEST)

# Display the outline
display_outline(outline.model_dump())

📝 Generating presentation outline...


### Phase 2: Build the Slideshow

Now we run the full workflow - for each slide in the outline:

1. **Search** - Query NASA Images API
2. **Select** - Researcher Agent picks best match
3. **Review** - Reviewer Agent validates the choice
4. **Retry/Judge** - If rejected, try again or use Judge Agent

In [30]:
from IPython.display import display, HTML

# Store events for analysis
all_events = []
final_slideshow = None

print("🚀 Starting multi-agent slideshow builder...\n")
print("="*60)

async for event in orchestrator.build_slideshow(USER_REQUEST):
    all_events.append(event)
    event_type = event.get("type", "")
    
    # Handle different event types
    if event_type == "phase":
        print(f"\n📍 {event['phase'].upper()}: {event['message']}")
        
    elif event_type == "outline_ready":
        print(f"\n📋 OUTLINE: {event.get('title', 'Untitled')}")
        print(f"   {event.get('narrative', '')[:80]}...")
        for slide in event.get('slides', []):
            print(f"   #{slide['position']} {slide['subject']} - {slide['topic']}")
            print(f"      Keywords: {slide.get('search_keywords', [])}")
        
    elif event_type == "slide_started":
        print(f"\n{'='*60}")
        print(f"🎬 SLIDE {event['position']}/{event['total']}: {event['subject']}")
        print(f"   Topic: {event['topic']}")
        
    elif event_type == "search_started":
        print(f"   🔍 Searching (attempt {event.get('attempt', '?')})...")
        
    elif event_type == "search_completed":
        print(f"   🔍 Query: '{event.get('query', '')}' → {event.get('result_count', 0)} results")
        if event.get('candidates'):
            for c in event['candidates'][:3]:
                print(f"      • {c.get('title', '')[:50]}")
        
    elif event_type == "search_retry":
        print(f"   ⚠️ {event.get('message', 'Retrying...')}")
        
    elif event_type == "image_selected":
        print(f"   📸 Selected: {event.get('title', '')}")
        print(f"      Reason: {event.get('reason', '')[:60]}")
        thumb = event.get('thumbnail_url')
        if thumb:
            display(HTML(f'<img src="{thumb}" style="max-width:200px; margin:5px 0;">'))
        
    elif event_type == "review_completed":
        status = "✅ APPROVED" if event.get('approved') else "❌ REJECTED"
        print(f"   {status}: {event.get('feedback', '')[:60]}")
        if event.get('search_suggestion'):
            print(f"      Suggestion: {event.get('search_suggestion')}")
        
    elif event_type == "slide_complete":
        slide = event.get('slide')
        if slide:
            img = slide.get('image', {})
            print(f"   ✨ COMPLETE: {img.get('title', 'No image')[:40]}")
            thumb = slide.get('thumbnail_url') or img.get('thumbnail_url')
            if thumb:
                display(HTML(f'<img src="{thumb}" style="max-width:300px; border:2px solid green; margin:5px 0;">'))
        else:
            print(f"   ❌ FAILED: No image found for this slide")
        
    elif event_type == "slideshow_complete":
        final_slideshow = event
        print(f"\n{'='*60}")
        print("🎉 SLIDESHOW COMPLETE!")
        print(f"   Total slides: {event.get('total_slides', 0)}")
        
    elif event_type == "error":
        print(f"   ❌ ERROR: {event.get('message', 'Unknown error')}")
        
    else:
        # Debug: show any other event types
        print(f"   [DEBUG] {event_type}: {str(event)[:80]}")

print(f"\n📊 Total events captured: {len(all_events)}")

🚀 Starting multi-agent slideshow builder...


📍 PLANNING: Creating presentation outline...

📋 OUTLINE: FULL PLANET VISIBLE: A Visual Tour of Our Solar System
   Explore the solar system in a visual journey, starting with the Sun and moving t...
   #1 Sun - Full view of the Sun showcasing its glowing surface and solar details
      Keywords: ['Sun', 'solar surface', 'solar disk']
   #2 Mercury - Full planetary disk view highlighting Mercury's cratered and rocky surface
      Keywords: ['Mercury', 'planet disk', 'Mercury full disk']
   #3 Venus - Full view of Venus showing its cloud-covered atmosphere
      Keywords: ['Venus', 'planet disk', 'Venus full disk']
   #4 Earth - Full globe view displaying Earth's blue oceans, white clouds, and continents
      Keywords: ['Earth', 'planet Earth', 'Earth full disk']
   #5 Mars - Full planet disk revealing Mars' rusty red surface details
      Keywords: ['Mars', 'planet disk', 'Mars full disk']
   #6 Jupiter - Full disk image capturing Jupiter’s

   [DEBUG] review_started: {'type': 'review_started', 'position': 1, 'nasa_id': 'PIA25628', 'title': 'Three
   ❌ REJECTED: The image is a composite using multiple telescope data inclu
      Suggestion: full Sun disc
   🔍 Searching (attempt 2)...
   🔍 Query: 'Sun' → 8 results
      • Sun Shines in High-Energy X-rays
      • The Sun Magnetic Field
      • Pulses from the Sun
   [DEBUG] selection_started: {'type': 'selection_started', 'position': 1, 'candidate_count': 8}
   📸 Selected: Pulses from the Sun
      Reason: This image presents a clear, full-disc view of the Sun, show


   [DEBUG] review_started: {'type': 'review_started', 'position': 1, 'nasa_id': 'PIA17669', 'title': 'Pulse
   ✅ APPROVED: The image shows a full disc view of the Sun, clearly display
   ✨ COMPLETE: Pulses from the Sun


   [DEBUG] slide_selected: {'type': 'slide_selected', 'position': 1, 'nasa_id': 'PIA17669', 'title': 'Pulse

🎬 SLIDE 2/8: Mercury
   Topic: Full planetary disk view highlighting Mercury's cratered and rocky surface
   🔍 Searching (attempt 1)...
   🔍 Query: 'Mercury Mercury' → 8 results
      • Mercury, Mercury! 
      • Revelations on Mercury
      • Breaking Mercury
   [DEBUG] selection_started: {'type': 'selection_started', 'position': 2, 'candidate_count': 8}
   📸 Selected: Mercury, Mercury!
      Reason: This image best matches the slide requirement as it clearly 


   [DEBUG] review_started: {'type': 'review_started', 'position': 2, 'nasa_id': 'PIA16908', 'title': 'Mercu
   ✅ APPROVED: The image shows a clear full planetary disk of Mercury as a 
   ✨ COMPLETE: Mercury, Mercury!


...

   [DEBUG] review_started: {'type': 'review_started', 'position': 7, 'nasa_id': 'PIA09005', 'title': 'Spell
   ✅ APPROVED: The image clearly shows Saturn as a full circular planet dis
   ✨ COMPLETE: Spellbinding Saturn


   [DEBUG] slide_selected: {'type': 'slide_selected', 'position': 7, 'nasa_id': 'PIA09005', 'title': 'Spell

🎬 SLIDE 8/8: Uranus
   Topic: Full circular disk of Uranus showcasing its pale blue-green color
   🔍 Searching (attempt 1)...
   🔍 Query: 'Uranus Uranus' → 8 results
      • GIF of Uranus' Magnetic Field
      • Uranus
      • Uranus Atmosphere
   [DEBUG] selection_started: {'type': 'selection_started', 'position': 8, 'candidate_count': 8}
   📸 Selected: Uranus - Final Image
      Reason: This image shows the full disk of Uranus as a pale blue-gree


   [DEBUG] review_started: {'type': 'review_started', 'position': 8, 'nasa_id': 'PIA00143', 'title': 'Uranu
   ❌ REJECTED: The image shows Uranus as a thin crescent rather than a full
      Suggestion: Uranus full disc
   🔍 Searching (attempt 2)...
   🔍 Query: 'Uranus' → 8 results
      • GIF of Uranus' Magnetic Field
      • Uranus
      • Uranus Atmosphere
   [DEBUG] selection_started: {'type': 'selection_started', 'position': 8, 'candidate_count': 8}
   📸 Selected: Uranus as seen by NASA Voyager 2
      Reason: This image shows a clear, full circular disk of Uranus as ca


   [DEBUG] review_started: {'type': 'review_started', 'position': 8, 'nasa_id': 'PIA18182', 'title': 'Uranu
   ✅ APPROVED: The selected image shows a full circular disk of Uranus with
   ✨ COMPLETE: Uranus as seen by NASA Voyager 2


   [DEBUG] slide_selected: {'type': 'slide_selected', 'position': 8, 'nasa_id': 'PIA18182', 'title': 'Uranu

📍 COMPLETE: Slideshow complete with 8 slides!

🎉 SLIDESHOW COMPLETE!
   Total slides: 8

📊 Total events captured: 95


### View the Final Slideshow

In [34]:
from IPython.display import display, HTML

if final_slideshow:
    print("="*60)
    print(f"🚀 {final_slideshow.get('title', 'Slideshow')}")
    print(f"   {final_slideshow.get('narrative', '')}")
    print("="*60)
    
    slides = final_slideshow.get('slides', [])
    print(f"\n📊 Total slides: {len(slides)}")
    
    for slide in slides:
        print(f"\n#{slide['position']} | {slide['subject']} - {slide['topic']}")
        img = slide.get('image', {})
        print(f"   📷 {img.get('title', 'No image')}")
        
        # Show image
        thumb = slide.get('thumbnail_url') or img.get('thumbnail_url')
        if thumb:
            display(HTML(f'''
            <div style="margin: 10px 0; padding: 10px; border: 1px solid #ccc; border-radius: 8px;">
                <img src="{thumb}" style="max-width: 400px; border-radius: 4px;">
                <p style="margin: 5px 0; color: #666;"><b>#{slide['position']}</b> {slide['subject']}</p>
            </div>
            '''))
        else:
            print(f"   ⚠️ No thumbnail URL")
else:
    print("⚠️ No slideshow was generated.")
    print("\n🔍 Debug - checking all_events for issues:")
    for e in all_events[-10:]:
        print(f"   {e.get('type')}: {str(e)[:100]}")

🚀 FULL PLANET VISIBLE: A Visual Tour of Our Solar System
   Explore the solar system in a visual journey, starting with the Sun and moving through each planet. Each slide features a full-view image of a planet or the sun as a circular orb, providing an iconic and clear perspective on each celestial body in our solar system.

📊 Total slides: 8

#1 | Sun - Full view of the Sun showcasing its glowing surface and solar details
   📷 Pulses from the Sun



#2 | Mercury - Full planetary disk view highlighting Mercury's cratered and rocky surface
   📷 Mercury, Mercury!



#3 | Venus - Full view of Venus showing its cloud-covered atmosphere
   📷 Venus Cloud Tops Viewed by Hubble



#4 | Earth - Full globe view displaying Earth's blue oceans, white clouds, and continents
   📷 Earth - Pacific Ocean



#5 | Mars - Full planet disk revealing Mars' rusty red surface details
   📷 Mars Full Disk Image from Viking Orbiter



#6 | Jupiter - Full disk image capturing Jupiter’s swirling cloud bands and iconic Great Red Spot
   📷 Jupiter Ahoy!



#7 | Saturn - Full planet and its rings seen from afar as a circular planet disk with ring shadows
   📷 Spellbinding Saturn



#8 | Uranus - Full circular disk of Uranus showcasing its pale blue-green color
   📷 Uranus as seen by NASA Voyager 2


## Key Takeaways

### Multi-Agent Workflow Patterns

1. **Separation of Concerns**: Each agent has a single responsibility
   - Planner → Structure
   - Researcher → Selection
   - Reviewer → Validation
   - Judge → Arbitration

2. **State Management**: All agents share a `SlideWorkflowState` that:
   - Tracks current progress
   - Records conversation history
   - Prevents duplicate selections

3. **Retry Loops**: The workflow supports:
   - Automatic retry on rejection
   - Max attempt limits
   - Fallback to Judge agent

4. **Event Streaming**: Real-time events enable:
   - Progress tracking
   - UI updates
   - Debugging/analysis

### When to Use Multi-Agent Workflows

✅ **Good Fit:**
- Tasks requiring multiple specialized skills
- Quality-critical processes needing review cycles
- Complex decisions benefiting from multiple perspectives

❌ **Overkill:**
- Simple Q&A or chat
- Single-step transformations
- Tasks where latency is critical